# **Scraping SatuSehat News Content**

Web scraping di sini menggunakan `BeautifulSoup` karena library ini memudahkan parsing dan navigasi HTML, serta ekstraksi data seperti teks dan judul artikel. `BeautifulSoup` digunakan bersama `requests` untuk mengambil konten web.

# Set Up

In [ ]:
!pip install pandas openpyxl beautifulsoup4 requests

In [ ]:
# Import Library
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
# Upload File
from google.colab import files
uploaded = files.upload()

satusehat_news_df = pd.read_excel(list(uploaded.keys())[0])

Saving link_berita_satusehat_cleaned.xlsx to link_berita_satusehat_cleaned.xlsx


In [ ]:
satusehat_news_df

,link,judul,tanggal,portal
0,https://cahayasiang.id/dinilai-peduli-lindungi...,"Dinilai Peduli Lindungi Para Pekerja, BPJS Ket...",2024-09-22,Cahaya Siang
1,https://memorandum.disway.id/read/114165/pemer...,Pemerintah Tingkatkan Layanan Haji dan Umrah M...,2024-09-15,Memorandum.co.id
2,https://www.bicaranetwork.com/kesehatan/295134...,Tutorial Cara Isi Formulir Satu Sehat Pasca Pu...,2024-09-06,Bicara Network
3,https://www.genpi.co/kesehatan/245752/bandara-...,Bandara I Gusti Ngurah Rai Wajibkan Penumpang ...,2024-09-06,GenPI.co
4,https://jakarta.akurat.co/lifestyle/1315048356...,Tok! Pemerintah Resmi Berlakukan Kembali Skrin...,2024-09-06,Akurat Jakarta
...,...,...,...,...
270,https://sehatnegeriku.kemkes.go.id/baca/umum/2...,Kemenkes Luncurkan QR Code untuk Aplikasi Pedu...,2021-07-01,Sehat Negeriku
271,https://www.suara.com/health/2021/01/18/114052...,Cara Mendapatkan Vaksin Covid-19 Gratis dari P...,2021-01-18,Suara.com
272,https://portalkudus.pikiran-rakyat.com/nasiona...,NIK Nakes Belum Terdaftar? Segera Kirim Email ...,2021-01-13,Portal Kudus
273,https://www.cnbcindonesia.com/tech/20201223171...,Pengumuman! Kini Bisa Konsultasi Kesehatan di ...,2020-12-23,CNBC Indonesia


# Web Scraping

In [ ]:
# Fungsi untuk menghapus teks iklan, "baca juga", dll.
def clean_content(content):
    # Hapus semua teks yang mengandung "baca juga", "iklan", atau yang sejenis
    content = re.sub(r'\b(Baca Juga|Iklan|ADVERTISEMENT|baca juga|IKLAN)\b', '', content, flags=re.IGNORECASE)
    return ' '.join(content.split())  # Hapus spasi berlebih, jadi satu baris

In [ ]:
# Fungsi untuk mengambil konten berita dan tags dari link
def get_article_content(link):
    try:
        response = requests.get(link)
        if response.status_code != 200:
            return None, None, "failed"  # Mengembalikan 3 nilai: content, tags, status

        # Menggunakan BeautifulSoup untuk parsing HTML
        soup = BeautifulSoup(response.content, 'html.parser')

        # Ambil semua paragraf
        paragraphs = soup.find_all('p')
        if not paragraphs:
            return None, None, "failed"  # Mengembalikan 3 nilai jika tidak ada paragraf

        # Gabungkan semua paragraf menjadi satu baris
        content = ' '.join([p.get_text() for p in paragraphs])
        content = clean_content(content)

        if len(content.strip()) == 0:
            return None, None, "failed"  # Mengembalikan 3 nilai jika konten kosong

        # Mengambil tags dari meta tag 'keywords' atau elemen lainnya
        meta_tags = soup.find('meta', attrs={'name': 'keywords'})
        if meta_tags and 'content' in meta_tags.attrs:
            tags = meta_tags['content']
        else:
            tags = "Tidak ada tags"

        return content, tags, "success"  # Mengembalikan 3 nilai sukses

    except Exception as e:
        print(f"Error mengambil konten dari {link}: {e}")
        return None, None, "failed"  # Mengembalikan 3 nilai jika ada error

# Menambah kolom baru untuk konten, tags, dan status
satusehat_news_df['content'] = None
satusehat_news_df['tags'] = None
satusehat_news_df['status'] = None

# Loop untuk mengambil isi berita dari setiap link
success_count = 0
failed_count = 0

In [ ]:
for index, row in satusehat_news_df.iterrows():
    link = row['link']

    # Coba mengambil artikel dengan BeautifulSoup
    content, tags, status = get_article_content(link)

    # Menyimpan hasil dalam DataFrame
    satusehat_news_df.at[index, 'content'] = content if content else "Tidak ada konten"
    satusehat_news_df.at[index, 'tags'] = tags if tags else "Tidak ada tags"
    satusehat_news_df.at[index, 'status'] = status

    # Menghitung keberhasilan dan kegagalan
    if status == "success":
        success_count += 1
    else:
        failed_count += 1

print(f"Jumlah artikel yang berhasil diambil: {success_count}")
print(f"Jumlah artikel yang gagal diambil: {failed_count}")

Error mengambil konten dari https://blitarkota.go.id/berita/qr-code-peduli-lindungi-masih-diajukan-ke-pemerintah-pusat-pemeriksaan-bukti-vaksin-wisata-mbk-dilakukan-manual: HTTPSConnectionPool(host='blitarkota.go.id', port=443): Max retries exceeded with url: /berita/qr-code-peduli-lindungi-masih-diajukan-ke-pemerintah-pusat-pemeriksaan-bukti-vaksin-wisata-mbk-dilakukan-manual (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1007)')))
Jumlah artikel yang berhasil diambil: 174
Jumlah artikel yang gagal diambil: 101


## Simpan Konten Berita ke dalam CSV dan Excel

In [ ]:
# Simpan DataFrame ke dalam CSV dan Excel
output_csv = 'satusehat_news_content.csv'
output_excel = 'satusehat_news_content.xlsx'

satusehat_news_df.to_csv(output_csv, index=False)
satusehat_news_df.to_excel(output_excel, index=False)

In [ ]:
satusehat_news_df

,link,judul,tanggal,portal,content,tags,status
0,https://cahayasiang.id/dinilai-peduli-lindungi...,"Dinilai Peduli Lindungi Para Pekerja, BPJS Ket...",2024-09-22,Cahaya Siang,Beranda » Dinilai Peduli Lindungi Para Pekerja...,Tidak ada tags,success
1,https://memorandum.disway.id/read/114165/pemer...,Pemerintah Tingkatkan Layanan Haji dan Umrah M...,2024-09-15,Memorandum.co.id,06 Okt 2024 06 Okt 2024 Polrestabes Surabaya 2...,"ketua fk patuh jatim,vaksin maningitis,satu se...",success
2,https://www.bicaranetwork.com/kesehatan/295134...,Tutorial Cara Isi Formulir Satu Sehat Pasca Pu...,2024-09-06,Bicara Network,BICARANETWORK.COM - Guna menanggulangi penyeba...,"mpox, Satu Sehat, cara isi formulir",success
3,https://www.genpi.co/kesehatan/245752/bandara-...,Bandara I Gusti Ngurah Rai Wajibkan Penumpang ...,2024-09-06,GenPI.co,GenPI.co - Bandara I Gusti Ngurah Rai Bali mew...,"Berita terkini mpox, cacar monyet, Monkeypox,...",success
4,https://jakarta.akurat.co/lifestyle/1315048356...,Tok! Pemerintah Resmi Berlakukan Kembali Skrin...,2024-09-06,Akurat Jakarta,Tidak ada konten,Tidak ada tags,failed
...,...,...,...,...,...,...,...
270,https://sehatnegeriku.kemkes.go.id/baca/umum/2...,Kemenkes Luncurkan QR Code untuk Aplikasi Pedu...,2021-07-01,Sehat Negeriku,"Bali, 1 Juli 2021 Kementerian Kesehatan (Kemen...",Tidak ada tags,success
271,https://www.suara.com/health/2021/01/18/114052...,Cara Mendapatkan Vaksin Covid-19 Gratis dari P...,2021-01-18,Suara.com,Suara.com - Seperti yang telah diketahui bersa...,"cara mendapatkan vaksin Covid-19 gratis, Vaksi...",success
272,https://portalkudus.pikiran-rakyat.com/nasiona...,NIK Nakes Belum Terdaftar? Segera Kirim Email ...,2021-01-13,Portal Kudus,Tidak ada konten,Tidak ada tags,failed
273,https://www.cnbcindonesia.com/tech/20201223171...,Pengumuman! Kini Bisa Konsultasi Kesehatan di ...,2020-12-23,CNBC Indonesia,"Jakarta, CNBC Indonesia - Pengguna Aplikasi Pe...","aplikasi pedulilindungi, kominfo, telemedis, k...",success


In [ ]:
# Menghapus baris yang memiliki status "failed"
satusehat_news_df_cleaned = satusehat_news_df[satusehat_news_df['status'] != 'failed']

In [ ]:
satusehat_news_df_cleaned

,link,judul,tanggal,portal,content,tags,status
0,https://cahayasiang.id/dinilai-peduli-lindungi...,"Dinilai Peduli Lindungi Para Pekerja, BPJS Ket...",2024-09-22,Cahaya Siang,Beranda » Dinilai Peduli Lindungi Para Pekerja...,Tidak ada tags,success
1,https://memorandum.disway.id/read/114165/pemer...,Pemerintah Tingkatkan Layanan Haji dan Umrah M...,2024-09-15,Memorandum.co.id,06 Okt 2024 06 Okt 2024 Polrestabes Surabaya 2...,"ketua fk patuh jatim,vaksin maningitis,satu se...",success
2,https://www.bicaranetwork.com/kesehatan/295134...,Tutorial Cara Isi Formulir Satu Sehat Pasca Pu...,2024-09-06,Bicara Network,BICARANETWORK.COM - Guna menanggulangi penyeba...,"mpox, Satu Sehat, cara isi formulir",success
3,https://www.genpi.co/kesehatan/245752/bandara-...,Bandara I Gusti Ngurah Rai Wajibkan Penumpang ...,2024-09-06,GenPI.co,GenPI.co - Bandara I Gusti Ngurah Rai Bali mew...,"Berita terkini mpox, cacar monyet, Monkeypox,...",success
9,https://newsindonesia.co.id/read/berita-sumene...,"Transformasi Digital, RSUD Moh Anwar Sumenep B...",2024-02-01,newsindonesia.co.id,© Copyright 2024 News Indonesia . All Right Re...,Tidak ada tags,success
...,...,...,...,...,...,...,...
267,https://www.beritamerdekaonline.com/2021/07/pe...,Perumahan tas Blok M & N Motto Satgas “Peduli ...,2021-07-21,Berita Merdeka Online,"Sidoarjo, beritamerdekaonline.com – Kegiatan m...",Tidak ada tags,success
268,https://www.idxchannel.com/economics/masih-bin...,Masih Bingung? Ini Tips Mudah Download dan Cet...,2021-07-12,IDX Channel,Scan this QR or download app from: IDXChannel ...,"IDX, BEI, Pasar modal, Vakinsasi",success
270,https://sehatnegeriku.kemkes.go.id/baca/umum/2...,Kemenkes Luncurkan QR Code untuk Aplikasi Pedu...,2021-07-01,Sehat Negeriku,"Bali, 1 Juli 2021 Kementerian Kesehatan (Kemen...",Tidak ada tags,success
271,https://www.suara.com/health/2021/01/18/114052...,Cara Mendapatkan Vaksin Covid-19 Gratis dari P...,2021-01-18,Suara.com,Suara.com - Seperti yang telah diketahui bersa...,"cara mendapatkan vaksin Covid-19 gratis, Vaksi...",success


In [ ]:
# Simpan DataFrame yang sudah dibersihkan ke dalam CSV dan Excel
output_csv = 'satusehat_news_content_cleaned.csv'
output_excel = 'satusehat_news_content_cleaned.xlsx'

satusehat_news_df_cleaned.to_csv(output_csv, index=False)
satusehat_news_df_cleaned.to_excel(output_excel, index=False)